In [1]:
# Import libaries
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import time


### Step 1: Create a soup object from the home page

In [2]:
url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/'
forscrapy = requests.get(url)
soup = BeautifulSoup(forscrapy.content, 'lxml')

In [3]:
soup.extract()

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Nutrition Information</title>
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>
</head>
<body>
<header>
<section class="container">
<nav class="navbar navbar-expand-lg navbar-light bg-light" role="navigation">
<a class="navbar-brand" href="/">Nutrition Information</a> </nav>
</section>
</header>
<main class="container" role="main">
<br/>
<div class="alert alert-danger">
        NOTE: This data is super old and rife with errors. It's meant for scraping practice only.
      </div>
<table class="table" id="restaurants">
<thead>
<tr>
<th>Name</th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<td>
<a href="restaurants/1.html">A&amp;W Restauran

In [4]:
print(soup.td)
print(soup.td.text)
#print(soup.td.href.text)
name = soup.find('a')

<td>
<a href="restaurants/1.html">A&amp;W Restaurants</a> </td>

A&W Restaurants 


In [5]:
name.attrs

{'class': ['navbar-brand'], 'href': '/'}

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [6]:
table = soup.find('table', {'id': 'restaurants'})
type(table) 

bs4.element.Tag

In [7]:
table = soup.find('table', {'id': 'restaurants'})

restaurants = []

for row in table.find_all('a'):
    resnames = {}
    resnames['name'] = row.text
    resnames['href'] = row.attrs['href'] 
    restaurants.append(resnames)
    
pd.DataFrame(restaurants).head()

,name,href
0,A&W Restaurants,restaurants/1.html
1,Applebee's,restaurants/2.html
2,Arby's,restaurants/3.html
3,Atlanta Bread Company,restaurants/4.html
4,Bojangle's Famous Chicken 'n Biscuits,restaurants/5.html


In [8]:
restaurants[:5]

[{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'},
 {'name': "Applebee's", 'href': 'restaurants/2.html'},
 {'name': "Arby's", 'href': 'restaurants/3.html'},
 {'name': 'Atlanta Bread Company', 'href': 'restaurants/4.html'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'href': 'restaurants/5.html'}]

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

1. retrieve website address and place into soup object url
2. create soup object for each restaruant

3. soup .find table class = table

4. t.body tr items name

5. td for category,calories,fat,carbs

In [9]:
#type(table)

In [10]:
#soup.find('tr')

In [11]:
#soup.find_all('tr')

In [12]:
restaurants[:5]

[{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'},
 {'name': "Applebee's", 'href': 'restaurants/2.html'},
 {'name': "Arby's", 'href': 'restaurants/3.html'},
 {'name': 'Atlanta Bread Company', 'href': 'restaurants/4.html'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'href': 'restaurants/5.html'}]

In [13]:
#for website in restaurants:
 #   print(website['href'])

In [23]:

rants= []

for website in restaurants:
    webstr = website['href']    
    pageurl = f'https://pages.git.generalassemb.ly/rldaggie/for-scraping/{webstr}'
    src = requests.get(pageurl)
    soup = BeautifulSoup(src.content, 'lxml')
    table = soup.find('table', {'class': 'table'})
 
    for row in table.find('tbody').find_all('tr'):
            name = soup.find('h3',{'class':'display-3'})
            resi = {} # dictionary container
            resi['itemname'] = row.find('td').text
            resi['Category'] = row.find_all('td')[1].text
            resi['Calories'] = row.find_all('td')[2].text
            resi['Fat'] = row.find_all('td')[3].text
            resi['Carbs'] = row.find_all('td')[4].text
            resi['res_name'] = name.text[1:]
            rants.append(resi) # append my dictionary to global list


retrieve website address and place into soup object url

create soup object for each restaruant 

soup .find table class = table

t.body tr items name

td for category,calories,fat,carbs





### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [24]:
df = pd.DataFrame(rants)

df.shape

(5131, 6)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [27]:
df.to_csv('/Users/j_b/ga_DSI_works/5.01-lab-webscraping/foodmenu.csv',index=False)